In [1]:
#-------- Import packages -------- #
import numpy as np 
import numpy.ma as ma
import pandas as pd 
import scipy
import scipy.stats as stats
from scipy.optimize import curve_fit
%matplotlib inline
import matplotlib.pyplot as plt
import time
import math
import itertools
import graph_tool.all as gt
import sys, argparse, csv, math
from dateutil.parser import parse 
from unidecode import unidecode 
import datetime 
import re 

In [ ]:
#-------- Agent-based model of social hierarchy: main script -------- #

#Set model parameters
delta = 0.2
alpha = 0.2
eta = 0.5
eps = 0.1

#Set simulation parameters
N = 1000
ic = 'Egal'
step=N     
T=1000*N
maxS = 2
Sav = maxS/2.
nr=10

#Initiate output arrays
S_av = np.zeros(N) 
m2 = np.zeros(T/step)
m2_av = np.zeros(T/step)
m3 = np.zeros(T/step)
m3_av = np.zeros(T/step)
m4 = np.zeros(T/step)
m4_av = np.zeros(T/step)
dirout = '/home/jhickey/outputdir/'

#Loop over nr realizations of the simulation
start_time=time.time()
for run in range(0,nr):   

    #Print progress to screen
    if run%(nr/10)==0: print "run =", run, "time elapsed =", np.round(time.time()-start_time,0)

    #Assign a status to each agent according to the initial condition
    if ic == 'Egal':
        S = np.ones(N)*(maxS/2.*N/N) #Egalitarian (delta-function) initial status
    elif ic == 'Unif':
        S = np.sort(np.random.random_sample(size=N)*maxS) #Random initial status, drawn from uniform distn
        
    #Loop over simulation time steps
    for t in range(0,T+1):    

        #Pick pair of agents randomly (np.random.choice() is too slow)
        pair=np.array([-1,-1])
        r = np.random.random_sample()
        pair[0]=int(np.floor(r*N))
        while pair[1] < 0:
            idx=0
            r = np.random.random_sample()
            val = int(np.floor(r*N))
            if  val != pair[0]: pair[1] = val

        #Assign higher status of pair to Sidx1 and lower status to Sidx2
        strengths = np.array([S[pair[0]],S[pair[1]]])        
        Sidx1 =  pair[np.argsort(strengths)[1]]
        S1 = S[Sidx1]
        Sidx2 = pair[np.argsort(strengths)[0]]
        S2 = S[Sidx2]   

        #Criterion for determining if fight takes place
        fight_occurs=False
        if ((S1-S2) < eta*Sav) or (np.random.random_sample() < eps): fight_occurs=True
        
        if fight_occurs == True:  
            
            #Calculate p
            p = 1./(1+(S2/S1)**alpha)                  
            r = np.random.random_sample()

            if r <= p: #Sidx1 (higher-status agent) wins fight
                S[Sidx1] += S2*delta
                S[Sidx2] -= S2*delta

            elif r > p: #Sidx2 (lower-status agent) wins fight       
                S[Sidx1] -= S1*delta
                S[Sidx2] += S1*delta    

        if t%step == 0: 
            i = t/step
            m2[i] = stats.moment(S,moment=2)
            m3[i] = stats.moment(S,moment=3)                         

    #Build up quantities to be averaged over the nr realizations
    m2_av += m2
    m3_av += m3                    
    S_av += np.sort(S)      

#Calculate averages over all realizations
m2_av = m2_av/float(nr)
m3_av = m3_av/float(nr)                        
S_av = S_av/float(nr)                 

#Save averaged variables S_av, m2_av, m3_av
fstring = '_N'+str(N)+'_alpha'+str(alpha)+'_delta'+str(delta)+'_eta'+str(eta)+'_eps'+str(eps)+'_maxS'+str(maxS)+'_ic'+ic+'_nr'+str(nr)+'_T'+str('%.3e' % T)+'_step'+str('%.e' % step)+'.npy'
np.save(dirout+'S_av'+fstring,S_av)
np.save(dirout+'m2_av'+fstring,m2_av)
np.save(dirout+'m3_av'+fstring,m3_av)

In [ ]:
#-------- Agent-based model of social hierarchy: Kolmogorov-Smirnov (KS) test of compatibility of household income distributions with exponential distribution -------- #

#Define function f: Root of f is maximum-likelihood parameter (T) for exponential distribution
def f(T, xl, xh, sumx):
    xl = p[0]
    xh = p[1]
    sumx = p[2]
    return T + (xl-xh)*np.exp((xl-xh)/T)/(1-np.exp((xl-xh)/T)) - sumx

#Define function to load-in and sort household income data and weights
def select_data(xlow, xhigh):
    #Load household income data and weights
    dirname = '/home/jhickey/Shared/IncomeData/IPUMS/IPUMS_USA/numpy_files/'
    y = np.load(dirname+'hhinc2015.npy')
    w = np.load(dirname+'hhinc2015_weights.npy')
    ypos = y[np.where(y>=0)]
    wpos = w[np.where(y>=0)]
    #Select data greater than xlow and smaller than xhigh
    data_and_weights = np.array([np.copy(ypos[np.where((ypos>xlow) & (ypos<xhigh))]), 
                                 np.copy(wpos[np.where((ypos>xlow) & (ypos<xhigh))])])
    #Sort by the "data" array, keeping the "weights" array tied to it        
    sort_idx = np.argsort(data_and_weights[0])
    data_and_weights = data_and_weights[:,sort_idx]
    #Define separate, properly sorted, data and weights arrays
    data = np.copy(data_and_weights)[0,:]
    weights = np.copy(data_and_weights)[1,:]
    return data, weights

### Perform KS test ### 

#Define set of lower cutoffs (xlows) and upper cutoffs (xhighs) over which to perform the KS test
xlow = 7e05
xhigh = 1.1e06
        
#Initialize variables to catch case of extremely poor fit (where MLE parameter cannot be obtained)
NaNpval = False
NaNthresh = 1e07

#Select data greater than xlow and smaller than xhigh
data, weights = select_data(xlow, xhigh)

#Define number of households in sample (N) and sum of weights (n)
N = float(data.shape[0])
n = float(np.sum(weights))

#Define parameters for newton method root search
sumx = 1/n*np.sum(weights*(data-xlow)) #include weights here        
p = (xlow, xhigh, sumx)
T0 = 1

#Do a first test to see if Tdata is obtainable
Tdata_check = scipy.optimize.newton(f, T0, args=p)
if Tdata_check > NaNthresh or Tdata_check < -NaNthresh:
    NaNpval=True    

#Find Tdata using newton's method        
Tdata = scipy.optimize.newton(f, T0, args=p)

#Calculate Fdata (theoretical CCDF)
Fdata = 1 - (1 - np.exp((xlow-data)/Tdata)) / (1 - np.exp((xlow-xhigh)/Tdata))        

#Calculate Fn (empirical CCDF)
Fn = []
for t in data: Fn.append(len(data[data>t])/N)
Fn = np.array(Fn)  

#Calculate Fw (weighted empirical CCDF)
Fw = []
for t in data: 
    indices = np.where(data>t)[0]    
    Fw.append(np.sum(weights[indices])/n)
Fw = np.array(Fw)                

#Find KS distance for the data
Ddata = np.max(abs(Fdata-Fw))    

## Find array of KS distances for synthetic datasets ##
N_trials = 1000
Dsynth = np.zeros(N_trials)        
for trial in range(0,N_trials):  

    #Generate synthetic data "synth" from exponential distribution with MLE parameter Tdata
    u = np.random.random(int(N))    
    synth = np.sort( xlow - Tdata * np.log(1 - (1-u)*(1-np.exp((xlow-xhigh)/Tdata))) ) 

    #Estimate Tsynth from the synth (this step needed to avoid bias, see doi:10.1142/S0219525909002131)
    sumx_synth = 1/N*np.sum(synth-xlow)            
    p = (xlow, xhigh, sumx_synth)
    T0 = 1  
    #Do a first test to see if Tsynth is obtainable
    Tsynth_check = scipy.optimize.newton(f, T0, args=p)
    if Tsynth_check > NaNthresh or Tsynth_check < -1*NaNthresh:
        NaNpval = True
        break             
    Tsynth = scipy.optimize.newton(f, T0, args=p)
    
    #Calculate Fsynth (theoretical CCDF)
    Fsynth = 1 - (1 - np.exp((xlow-synth)/Tsynth)) / (1 - np.exp((xlow-xhigh)/Tsynth))

    #Calculate Fn_synth (empirical CCDF)
    Fn_synth = 1. - np.arange(1,N+1)/N

    #Find Dsynth (compared to Fw)
    Dsynth[trial] = np.max(abs(Fsynth-Fn_synth))
    
if NaNpval == False: pval = len(np.where(Dsynth > Ddata)[0])/float(len(Dsynth))
else: pval = float('NaN') 
        
### Plot PDF of household income dist'n and fit of exp. dist'n with lower and upper cutoffs ###

fig = plt.figure(figsize=(11,5))
ax1 = fig.add_subplot(121) #log-linear axis scale
ax2 = fig.add_subplot(122) #log-log axis scale
   
#Plot PDF of household income data
nbins=200
hist, be = np.histogram(data, bins=np.logspace(0,np.log10(np.max(data)),nbins), density=True, weights=weights)
xh = np.logspace(0,np.log10(np.max(data)),len(hist))
ax1.plot(xh,np.log10(hist), '.-', label='data', linewidth=2)  
ax2.plot(np.log10(xh),np.log10(hist), '.-', label='data', linewidth=2)

#Plot PDF of theoretical fit (using parameter Tdata)
yfit = 1./Tdata*np.exp((xlow-data)/Tdata)/(1-np.exp((xlow-xhigh)/Tdata))    
ax1.plot(data,np.log10(yfit), '-r', label='fit, p-val = '+str(np.round(pval,2)), linewidth=2)
ax2.plot(np.log10(data),np.log10(yfit), '-r', label='fit', linewidth=2)

#Set the axis limits, axis labels, etc. on the figure
ax1.set_xlim(xlow,xhigh)
ax2.set_xlim(np.log10(xlow),np.log10(xhigh))
ax1.legend(loc='upper right')
ax1.set_xlabel('$S$')
ax1.set_ylabel('$\log_{10}[p(S)]$')
ax2.set_xlabel('$\log_{10}[S]$')
ax2.set_ylabel('$\log_{10}[p(S)]$')
ax1.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax2.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.tight_layout()

In [ ]:
#-------- Legal citation networks: Obtain temporal clustering of judgments by statistical inference of a generative model, using the expectation-maximization (EM) algorithm. (This is an application of the mixture model described in Section 2 of the paper by Leicht et al. at https://doi.org/10.1140/epjb/e2007-00271-7) -------- #

#Load graph from saved
lawtype = 'FAM'
dirname = '/home/jhickey/Shared/Citn_ntwk/data/'+lawtype+'/saved_ntwks/'
fname = lawtype+'_ntwk_20180901.xml.gz'
g = gt.load_graph(dirname+fname)

#Assign "internal vertex properties" to regular vertex properties 
citn = g.vp.citn_int
date = g.vp.date_int
crt = g.vp.crt_int
ltype = g.vp.ltype_int
title = g.vp.title_int
tags = g.vp.tags_int
crtrgnlvl = g.vp.crtrgnlvl_int

#Assign internal edge properties to regular edge properties 
treatment = g.ep.treatment_int
depth = g.ep.depth_int

#Remove parallel edges, if any exist
gt.remove_parallel_edges(g)

#Filter to keep only vertices of ltype
g.set_vertex_filter(ltype)

#Filter to keep only vertices up to current_year
current_year=2015
g = gt.GraphView(g, vfilt=lambda v: date[v].year <= current_year)
g = gt.GraphView(g, vfilt=gt.label_largest_component(g,directed=False))

#Create dataframe to time-order the nodes in g
datelist=[]
vlist=[]
degslist=[]
for v in g.vertices(): 
    vlist.append(int(v))
    datelist.append(date[v].year)
    degslist.append(int(v.in_degree()))
df = pd.DataFrame(data={'v': vlist, 'date': datelist, 'indegs': degslist})
df=df.sort(columns='date').set_index(np.arange(df.shape[0]))

#Define graph h, to be operated on
h = gt.Graph(g)
h.set_reversed(True)

#Initiate variables
start_year = df.date.iloc[0] #earliest year of a judgment in network h
end_year = current_year+1 
n = h.num_vertices() #number of nodes
T = end_year-start_year #number of years
z = np.zeros((n,T)) #number of citations from each judgment to each year

#Determine z(i,t)
for tup in df.v.iteritems():
    i = df[df['v'] == tup[1]].index[0] #index indicating citing node 
    v = h.vertex(tup[1])
    for e in v.in_edges():
        u = e.source()
        idx = df[df['v'] == int(u)].index[0] #index indicating cited node        
        t = df.date.iloc[idx]-start_year
        z[i,t] += 1
k = np.sum(z, axis=1) #for use in calculation of theta_denom (below)

#Initialize variables for EM procedure
n_runs = 10 #average over this many runs (realizations)
c_arr = np.arange(1,6) #c is the number of clusters 

for cidx, c in enumerate(c_arr):        
        
    q_prev = np.zeros((n,c)) #q array from previous iteration
    theta_prev = np.zeros((c,T)) #theta array from previous iteration
    ELs = np.zeros(n_runs)
    H = np.zeros(n_runs)    
    
    for run in range(0,n_runs):
               
        #Initialize pi and theta
        pi = 1/float(c)+np.random.random_sample(c)/10 #add random increment to uniform initial guess 
        pi = pi/sum(pi) #normalize
        theta = 1/float(T)+np.random.random_sample((c,T))/10 #add random increment to uniform init. guess 
        theta = theta/np.sum(theta,axis=1) #normalize

        #Initialize variables        
        q = np.zeros((n,c))        
        q_old = np.zeros((n,c))
        beta = np.zeros((n,c))        
        beta_max = np.zeros(n)  
        
        #Iterate q, theta, and pi until convergence in q
        n_iter = 10000
        start_time = time.time()
        for m in range(0,n_iter):
        
            #Compute beta by matrix multiplication (define custom "dot product")            
            beta = np.sum(np.log(theta[...,np.newaxis]**np.transpose(z)[np.newaxis,...]),axis=1) 
            beta = np.transpose(beta)        
            
            #Determine beta_max
            beta_max = np.max(beta,axis=1)                    
                
            #Define cutoff (small value that will not cause underflow when exponentiated)
            cutoff = np.log(1e-320)
                        
            #Compute numerator of q
            q_numer = np.transpose(np.subtract(np.transpose(beta),beta_max))
            q_numer[q_numer < cutoff] = float('-inf')
            q_numer = pi*np.exp(q_numer)
            #Compute denominator of q
            q_denom = np.sum(q_numer,axis=1)
            #Compute q
            q = np.transpose(np.transpose(q_numer)/q_denom)
                        
            #Check q for convergence 
            if np.allclose(q,q_old,rtol=1e-5) == True: break          
                
            #Check for NaN in q 
            if True in np.isnan(q): 
                print "NaN found 2"
                raise SystemExit
                break
                    
            #Compute pi and theta
            pi = 1/float(n)*np.sum(q,axis=0)
            theta_numer=np.dot(np.transpose(q),z)
            theta_denom = np.sum(np.transpose(np.tile(k,(c,1)))*q,axis=0)
            theta = np.transpose(np.transpose(theta_numer)/theta_denom)
                       
            #Copy q to q_old, will use to check for convergence in next iteration
            np.copyto(q_old,q)

        #Once convergence reached, print following
        print "m =", m, ' elapsed time =', time.time()-start_time                 
                              
        #Save each q output and theta_output            
        fsave='_c'+str(c)+'_run'+str(run)+'_'+lawtype+'_y1'+str(start_year)+'_y2'+str(current_year)+'.npy'
        np.save('/home/jhickey/outputdir/'+'q'+fsave,q)
        np.save('/home/jhickey/outputdir/'+'theta'+fsave,theta)

In [ ]:
#-------- Legal citation networks: Pre-process data and build graph with graph-tool package -------- #

#Define function used to remove accented characters from strings
def convert_string(string):
    try:
        string.decode('utf-8')
        out = unidecode(string.decode('utf-8')) #input string is UTF-8
    except UnicodeError:
        try:
            string.decode('latin-1')
            out = unidecode(string.decode('latin-1')) #input string is Latin-1
        except UnicodeError:
            print "string is not UTF-8 or Latin-1"
    return out

#Initiate graph "g"
g = gt.Graph()
#Define vertex properties: 
citn=g.new_vertex_property("string") #citation of judgment
crt=g.new_vertex_property("string") #court that issued judgment
date=g.new_vertex_property("object") #date of judgment (datetime format)
title=g.new_vertex_property("string") #title of judgment
ltype=g.new_vertex_property("bool") #used to filter judgments by area of law
tags = g.new_vertex_property("object") #list of topic tags attached to the judgment
crtrgnlvl = g.new_vertex_property("string") #e.g. "ON-Tr" for Ontario, Trial
#Define edge properties:
treatment = g.new_edge_property("string") #treatment 
depth = g.new_edge_property("int") #depth

### Main Build-ntwk code ###
#Each judgment in the "cited_all.Citations" series (loaded-in below) has a corresponding .csv file containing data on the judgments that cite it. This code builds the citation network from these .csv files as follows: for each "cited" judgment, open up its .csv file and assign links stemming from its "citing" judgments, creating new nodes each time a cited or citing judgment occurs that does not yet have a corresponding node in the network.

#Choose dataset to use
lawtype = 'DFM' 
dirname = '/home/jhickey/Shared/Citn_ntwk/data/'+lawtype+'/'

#Read-in Title, Date, Citn for cited judgments (Carswell citation only)
f = dirname+'lists_final/'+lawtype+'_cited_carswell.csv'
cited_carswell = pd.read_csv(f, skiprows=1, names=['Title','Date','Citation_1','Citation_2'])
#Read-in Title, Date, Citn for cited judgments (all citations: Carswell and non-Carswell) 
f = dirname+'lists_final/'+lawtype+'_cited_all.csv'
cited_all = pd.read_csv(f,skiprows=1,names=['Title','Date','Citations'])

#Add a semi-colon to end of every entry in "cited_all.Citations", to avoid error noted 2015-10-22
cited_all.Citations.str.lower()+';'

#Define replace_dict for cases where current_citing needs to be changed from Citation_2 to Citation_1
replace_dict = {}
for i in range(len(cited_carswell.Citation_2)):
    if type(cited_carswell.Citation_2[i]) == str:
        if cited_carswell.Citation_1[i].lower() not in replace_dict: 
            replace_dict[cited_carswell.Citation_2[i].lower()] = cited_carswell.Citation_1[i].lower()            

#Open file containing citations for SDST_dict ("SDST" refers to judgments with the same date and same title, which need to be checked manually to determine if they are duplicates or not)
f = dirname+'lists_final/'+lawtype+'_SDST_dictionary.csv'
SDST_entries = pd.read_csv(f,skiprows=1,names=['Favoured','Unfavoured'])

#Complete replace_dict by adding in SDST_entries not already present in replace_dict
for i in range(0,len(SDST_entries.Favoured)):
    if SDST_entries.Unfavoured[i] not in replace_dict:
        replace_dict[SDST_entries.Unfavoured[i]] = SDST_entries.Favoured[i] 

#nan_citations list is used to store data for judgments (or other entries, e.g. "CED") appearing in .csv files with no citation (read-in as 'NaN')
nan_citations = []

#Count how many times crt-level gets filled in 
crt_fill_count = 0

#Loop over all cited judgments (each one has its own .csv file of citing judgments)
for idx_cited in range(0,len(cited_carswell.Citation_1)):
    
    #Move to current judgment, for which a .csv file exists
    current_cited = cited_carswell.Citation_1[idx_cited].lower()
    
    #Correct current_cited if .csv file happens to contain "Unfavoured" replace_dict entry.
    if current_cited in replace_dict: current_cited = replace_dict[current_cited]        
    
    #Check to see if a vertex already exists for this judgment, and if not add new vertex
    x=gt.find_vertex(g,citn,current_cited) #x is a list of <class 'graph_tool.libgraph_tool_core.Vertex'>
    if len(x) == 0:
        #Add vertex
        vcurrent_cited = g.add_vertex() #index of new vertex representing "current_cited" judgment
        #Assign vertex properties
        citn[vcurrent_cited] = current_cited
        title[vcurrent_cited] = cited_carswell.Title[idx_cited]
        #Put date in proper format
        dt_cited = parse(cited_carswell.Date[idx_cited]).isoformat()[:10]
        date[vcurrent_cited] = datetime.datetime.strptime(dt_cited, "%Y-%m-%d")
         
    #If vertex for this current_cited judgment already exists, then set vcurrent_cited appropriately
    if len(x) == 1: vcurrent_cited = x[0]
        
    #Open file containing .csv spreadsheet and read-in data
    fname_csv = dirname+lawtype+'_csv_all/'+current_cited+'.csv'
    #Two lines below needed b/c of "skipfooter" error (instead of skipfooter in pd.read_csv, use nrows)
    f_csv = open(fname_csv,'r')
    num_lines = sum(1 for line in open(fname_csv))
    
    citing_refs=pd.read_csv(fname_csv, skiprows=1, nrows=num_lines-2, index_col=False, usecols=[0, 3, 5, 6, 7, 9, 11], names = ['Treatment', 'Title', 'Primary_Cite', 'Parallel_Cites', 'Court_or_Jurisdiction', 'Date', 'Depth'])

    #Loop over all the entries in the current .csv file
    for idx_citing in range(len(citing_refs.Primary_Cite)):
        
        current_citing = citing_refs.Primary_Cite[idx_citing]
        
        #Store judgments for which current_citing = 'NaN' and continue on to next idx_citing value
        if type(current_citing) is not str:
            if math.isnan(current_citing) == True: 
                nan_citations.append([current_cited, citing_refs.Title[idx_citing], citing_refs.Court_or_Jurisdiction[idx_citing], citing_refs.Parallel_Cites[idx_citing]])
                continue
            else: 
                print "Error 1"
                #Stop program
                raise SystemExit
                    
        #Replace current_citing with carswell version of citation, if possible
        y = cited_all.Citations.str.contains(re.escape(current_citing))
        if len(np.where(y==True)[0]) > 0:     
            l = cited_all.Citations.iloc[y[y==True].index[0]].split(';')
            current_citing = [val.strip().lower() for val in l if 'Carswell' in val][0]
                
        #Make sure current_citing is all lower-case 
        #(do after above "NaN"-related code b/c .lower() can't be applied to non-strings)
        current_citing = current_citing.lower()                
                
        #Correct current_citing if .csv file contains Citation_2
        if current_citing in replace_dict: current_citing = replace_dict[current_citing]            
            
        #Check if vertex for this current_citing judgment already exists, and if not add new vertex 
        x=gt.find_vertex(g,citn,current_citing)
        if len(x) == 0:
            #Add vertex 
            vcurrent_citing = g.add_vertex()
            #Assign vertex properties
            citn[vcurrent_citing] = current_citing
            title[vcurrent_citing] = citing_refs.Title[idx_citing]
            crt[vcurrent_citing] = citing_refs.Court_or_Jurisdiction[idx_citing]
            #Handle dates with only year value (these are read-in as "int" instead of "str")
            #Make month and day Jan. 01 by convention
            if type(citing_refs.Date[idx_citing]) is not str and citing_refs.Date[idx_citing] > 1800 and citing_refs.Date[idx_citing] < 2000:
                citing_refs.Date[idx_citing] = str(citing_refs.Date[idx_citing]) + "-01-01" 
            dt_citing = parse(citing_refs.Date[idx_citing]).isoformat()[:10] #put date in proper format
            date[vcurrent_citing] = datetime.datetime.strptime(dt_cited, "%Y-%m-%d")
        
        #If vertex for this current_citing judgment already exists, set vcurrent_citing appropriately
        if len(x) == 1: vcurrent_citing = x[0]
        
        #Assign "crt" vertex property if empty (b/c this inf. not directly avail. for cited jdgmnts)
        if crt[vcurrent_citing] == "": 
            crt[vcurrent_citing] = citing_refs.Court_or_Jurisdiction[idx_citing]
            crt_fill_count = crt_fill_count + 1
    
        #Add edge from vertex "current_citing" to vertex "current_cited"
        e = g.add_edge(vcurrent_citing, vcurrent_cited)
        
        #Assign edge properties to the new edge
        treatment[e] = citing_refs.Treatment[idx_citing]
        depth[e] = citing_refs.Depth[idx_citing]
        
#Replace accented characters
for v in g.vertices():
    crt[v] = convert_string(crt[v])
    title[v] = convert_string(title[v])
    crtrgnlvl[v] = convert_string(crtrgnlvl[v])        